In [1]:
import os
import geopandas as gpd
import numpy as np
from glob import glob
from multiprocessing.dummy import Pool as ThreadPool

from pySnowRadar import snowradar, picklayers
np.seterr(invalid='ignore', divide='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
can = world.loc[world['name'] == 'Canada']

In [3]:
radar_path = './data/nsidc/'
radar_files = [y for x in os.walk(radar_path) for y in glob(os.path.join(x[0], '*.nc'))]
land_flag = [can['geometry'].intersects(snowradar.SnowRadar(file, l_case='meta').poly) for file in radar_files]
radar_files = [radar_files[i] for i in [i for i, x in enumerate(land_flag) if not x.bool()]]

In [4]:
def process_snowradar(file):
    print('Processing: ' + file)
    radar_dat = snowradar.SnowRadar(radar_files[0], l_case='full')
    radar_dat.surf_bin, radar_dat.surface = radar_dat.get_surface()
    radar_bounds = radar_dat.get_bounds(5)
    radar_sub = radar_dat.data_radar[radar_bounds[1]: radar_bounds[0], :]
    radar_dat.calcpulsewidth() 
    
    b_as, b_si = np.apply_along_axis(
        picklayers.picklayers, 
        0, 
        radar_sub,
        radar_dat.n2n,
        radar_dat.dfr,
        1.238066467117174
    )
    result_df = gpd.pd.DataFrame({'Lat': radar_dat.lat,
                                  'Lon': radar_dat.lon,
                                  'b_as': b_as, 
                                  'b_si': b_si})
    return result_df

In [5]:
def process_parallel(file, threads=4):
    pool = ThreadPool(threads)
    results = pool.map(process_snowradar, file)
    pool.close()
    pool.join()
    return results

In [ ]:
output = process_parallel(radar_files)

Processing: ./data/nsidc\IRSNO1B_20160419_04_001_deconv.ncProcessing: ./data/nsidc\IRSNO1B_20160419_04_002_deconv.nc
Processing: ./data/nsidc\IRSNO1B_20160419_04_003_deconv.nc

Processing: ./data/nsidc\IRSNO1B_20160419_04_004_deconv.nc
